In [26]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import Geohash
from build import TreeBuilding
from decision import haversine
import pickle

%matplotlib inline

In [27]:
tree = pickle.load( open( "tree_10_new.p", "rb" ) )

In [28]:
def preorder(tree, points):
    if tree is not None:
        if tree.decision_point is not None:
            preorder(tree.left, points)
            preorder(tree.right, points)
            points.append(list(tree.decision_point))
            return points
        
def get_lats_lons(points):
    lats = [point[1] for point in points]
    lons = [point[0] for point in points]
    return lats, lons

In [41]:
decision_points = []
decision_points = preorder(tree.root, decision_points)

In [42]:
lats, lons = get_lats_lons(decision_points)

In [31]:
def order_left(tree, points):
    if tree is not None:
        if tree.decision_point is not None:
            #points.append(list(tree.decision_point))
            preorder(tree.left, points)
            #preorder(tree.right, points)
            points.append(list(tree.decision_point))
            return points

In [36]:
decision_points = []
decision_points = order_left(tree.root, decision_points)

In [37]:
len(decision_points)

392

In [38]:
lats, lons = get_lats_lons(decision_points)

In [44]:
import gmplot

gmap = gmplot.GoogleMapPlotter(41.1496100, -8.6109900 , 12)

#gmap.plot(lats, lons, 'cornflowerblue', edge_width=5)
#gmap.scatter(heat_lats, heat_lons, '#3B0B39', size=10, marker=False)
gmap.scatter(lats, lons, '#000000', size=15, marker=False)
#gmap.scatter(end_lats, end_lons, '#8B0000', size=10, marker=False)

#gmap.heatmap(heat_lats, heat_lons)

gmap.draw("plot10_new_left.html")

It seems I need two different loops:
1. To print out all nodes
2. To print out all edges

How to specify the size of the each node/edge??

It goes without saying, I need to know the number of trajectories in each node.

So, the idea: 
- the size of the node correspond to the number of trajectories in this node
- the size of the edge edge(p1,p2) correspond to the number of trajectories in p2

Осталось только понять величину максимальную

Some sort of order